## **Module**

In [1]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [2]:
os.chdir("/home/wmbio/WORK/gitworking/Multi-omics-intergration")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration'

In [3]:
# PATH
CANCER_TYPE = "BRCA"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
RDATA_PATH = os.getcwd() + "/RAW_DATA/GDC_PREPARE/"
RAW_PATH = os.getcwd() + "/RAW_DATA/"

METHOD = 'deseq2'
LOGFC=1
FDR=0.05

In [4]:
# Load Validation score
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'BRCA_validation.csv', usecols=col)

# Q3 value
SILHOUETTE = group_score.Silhouette.quantile(.7)
RNA_ANOVA = group_score.RNA_ANOVA_F1.quantile(.7)
RNA_RF = group_score.RNA_RF_F1.quantile(.7)
MIRNA_ANOVAR = group_score.miRNA_ANOVA_F1.quantile(.7)
MIRNA_RF = group_score.miRNA_RF_F1.quantile(.7)
MT_ANOVAR = group_score.Methylation_ANOVA_F1.quantile(.7)
MT_RF = group_score.Methylation_RF_F1.quantile(.7)

# Condition for Filtering
filter_cond = (group_score['Silhouette'] >= SILHOUETTE) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > RNA_ANOVA) | (group_score['RNA_RF_F1'] > RNA_RF)) & \
              ((group_score['miRNA_ANOVA_F1'] > MIRNA_ANOVAR) | (group_score['miRNA_RF_F1'] > MIRNA_RF)) & \
              ((group_score['Methylation_ANOVA_F1'] > MT_ANOVAR) | (group_score['Methylation_RF_F1'] > MT_RF))

In [5]:
group_score = group_score[filter_cond].sort_values(["Silhouette"], ascending = (False))

In [6]:
bestSubgroup = group_score.FILENAME.to_list()

In [7]:
if len(bestSubgroup) > 100:
    random.seed(331)
    bestSubgroup = random.sample(bestSubgroup, k=100)

In [8]:
DEG_CHECK = "_".join([CANCER_TYPE, METHOD.upper(), bestSubgroup[0]]) + ".txt"

In [9]:
# DEA
dea_result = list()
for best_group in bestSubgroup:
    
    DEG_CHECK = "_".join([CANCER_TYPE, METHOD.upper(), best_group]) + ".txt"
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "/" + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    if os.path.isfile(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK):
        deg_list = pd.read_csv(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK, sep = "\t")
    else :
        # DEG Extraction
        deg_list = deg_extract(log_fc=LOGFC, fdr=FDR,
                       cancer_type=CANCER_TYPE, 
                       sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                       file_name=best_group,
                       rdata_path=RDATA_PATH,
                       method=METHOD,
                       batch_removal=True,
                       raw_path = RAW_PATH)
    
    dea_result.append(deg_list)

R[write to console]: Error in value[[3L]](cond) : 
  GDC server down, try to use this package later



RRuntimeError: Error in value[[3L]](cond) : 
  GDC server down, try to use this package later


In [40]:
# Filter DEA
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)
elif METHOD == 'deseq2' :
    dea_combine = list(map(lambda d : d[["row", "log2FoldChange"]], dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'inner'), dea_combine)

# median & mean
dea_combine["SubGroup-log2FC_median"] = dea_combine.iloc[:, 1:].median(axis=1)
dea_combine["SubGroup-log2FC_mean"] = dea_combine.iloc[:, 1:].mean(axis=1)

* **NT vs TP DEA(Deseq2)**

In [11]:
nt_tp_deseq2 = deg_extract_normal(log_fc=0, pvalue=0.25, cancer_type=CANCER_TYPE, 
                                  rdata_path=RDATA_PATH, deg_path=DEG_PATH, batch_removal=True)

nt_tp_deseq2_col = nt_tp_deseq2[['row', 'log2FoldChange', 'pvalue']]
nt_tp_deseq2_col.columns = ['gene', 'NT-TP_log2FoldChange', 'pvalue']

Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  

* **Write Result**

In [12]:
pd.merge(left=dea_combine, right=nt_tp_deseq2_col, left_on='gene', right_on='gene', how = 'left').to_csv('temp.csv', index = False)